<a href="https://colab.research.google.com/github/prince24-web/Google-colabs/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.78
    Uninstalling langchain-core-0.3.78:
      Successfully uninstalled langchain-core-0.3.78


In [ ]:
%pip install -qU "langchain[google-genai]"

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai", google_api_key=GOOGLE_API_KEY)


In [ ]:
from inspect import classify_class_attrs
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template("""
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
""")

class Classification(BaseModel):
  sentiment: str = Field(..., enum=["happy", "neutral", "sad"]),
  aggressiveness: int = Field(description="describes how aggressive the statement is, the higher the number the more aggressive", enum=[1,2,3,4,5]),
  language: str = Field(..., enum=["english", "italian", "dutch", "spanish", "French"])

# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [ ]:
input = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": input})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='Positive', aggressiveness=(FieldInfo(annotation=NoneType, required=True, description='How aggressive the text is on a scale from 1 to 10'),), language='Spanish', translation='I am incredibly happy to have met you! I think we will be very good friends!')

In [ ]:
input = "Goedemorgen, hoe gaat het"
prompt = tagging_prompt.invoke({"input": input})
response = structured_llm.invoke(prompt)

response.model_dump()

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=(FieldInfo(annotation=Non...entiment of the text'),), input_type=tuple])
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [input_value=(FieldInfo(annotation=Non...a scale from 1 to 10'),), input_type=tuple])
  return self.__pydantic_serializer__.to_python(


{'sentiment': (FieldInfo(annotation=NoneType, required=True, description='The sentiment of the text'),),
 'aggressiveness': (FieldInfo(annotation=NoneType, required=True, description='How aggressive the text is on a scale from 1 to 10'),),
 'language': 'Dutch',
 'translation': 'Good morning, how are you'}

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
""")

structured_llm = llm.with_structured_output(Classification)

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
structured_llm.invoke(prompt)

Classification(sentiment='positive', aggressiveness=(FieldInfo(annotation=NoneType, required=True, description='describes how aggressive the statement is, the higher the number the more aggressive', json_schema_extra={'enum': [1, 2, 3, 4, 5]}),), language='spanish')